In [38]:
import os, config, itertools, re, sys
sys.path.append(os.path.join(os.getcwd(), os.pardir, os.pardir))
import nutools

In [41]:
years = ['2012', '2017']
regions = ['NE', 'SW']
modules = ['A', 'B']

In [45]:
# Parse XSPEC output for the values we are interested in.
# This is basically hardcoded to return those values.
def parseflux(log):
    try: 
        readpars = False
        pars = []
        redchisq = []
        for line in log:
            if 'Reduced chi-squared' in line:
                redchisq.append(float(re.findall(nutools.number, line)[0]))
            if readpars:
                par = re.findall(nutools.number, line)
                if len(par) == 5:
                    if par[0] is '3':
                        fluxmin = float(par[1])
                        flux = fluxmin - float(par[3])
                        fluxmax = float(par[2])
                else:
                    readpars = False
            if 'Parameter' in line:
                readpars = True
        return [redchisq[0], redchisq[1], fluxmin, flux, fluxmax]
    except:
        return [None]

In [54]:
# conduct native xspec analysis by executing a prewritten 
# template script and injecting the data to load

# xspec template analysis we want to conduct
analysis_file = config.EXPERIMENT_PATH + '/scripts/tbabsapecgauspo310.xcm'

os.chdir(config.PRODUCT_PATH) # xspec must be executed where the data is

for year, module, region in list(itertools.product(years, modules, regions)):
    stem = year + '_' + module + '_' + region
    infile = stem + '_bin.pha'
    if not os.path.isfile(infile): continue
        
    with open(analysis_file) as analysis:
        with open(config.PRODUCT_PATH + '/' + 'tmp.xcm', 'w') as tmp:
            for line in analysis:
                if 'INFILE' in line: line = line.replace('INFILE', infile)
                tmp.write(line)
            tmp.write('exit\n')
            tmp.close()
        analysis.close()
    # execute terminal command `xspec tmp.xcm` (special jupyter syntax) and store output in log
    log = !xspec tmp.xcm
    print(stem + ',' + ','.join([str(i) for i in parseflux(log)])) # format to be pastable into a spreadsheet for further analysis
    
os.remove('tmp.xcm')
os.chdir(config.EXPERIMENT_PATH)

2012_A_NE,1.292,1.1868,-11.719,-11.71092536,-11.703
2012_A_SW,1.045,0.9603,-11.9756,-11.9641857,-11.9531
2012_B_NE,1.12,1.024,-11.7184,-11.7051438,-11.6922
2012_B_SW,1.056,0.9608,-11.9459,-11.9283746,-11.9115
2017_A_NE,0.9499,0.8705,-11.6857,-11.6756945,-11.6659
2017_A_SW,0.9747,0.8943,-11.8587,-11.8466611,-11.8349
2017_B_NE,0.8691,0.7308,-11.7235,-11.6969944,-11.672
2017_B_SW,1.064,0.8234,-11.9418,-11.9070131,-11.8748
